# Multiple input single output model


In [1]:
from src.loaders.data_loader import load_train_data
from src.loaders.window_generator import WindowGenerator
from src.preprocessors.preprocessors import split_train_valid_test
from src.preprocessors.preprocessors import apply_standard_scale
from src.model.multiple.dense import MultipleDense
from src.preprocessors.add_columns import add_sin_cos

df = load_train_data()
df = add_sin_cos(df, "Hour")
n = len(df)
train_df, valid_df, test_df = split_train_valid_test(df, [0.7, 0.2, 0.1])
scaled_train_df, scaled_valid_df, scaled_test_df = apply_standard_scale(train_df, valid_df, test_df)


ratio should be 3 length list, example: [0.6, 0.3, 0.1]
sum of ration should be 1, example: [0.7, 0.2, 0.1]
shape of train, valid, test: (36792, 11), (10511, 11), (5257, 11)


In [ ]:
from src.trainers import compile_and_fit_with_pinball_loss

valid_performance = {}
performance = {}

conv_window_scaled = WindowGenerator(
    scaled_train_df,
    scaled_valid_df,
    scaled_test_df,
    input_width=48,
    label_width=1,
    shift=1,
    label_columns=["TARGET"]
)
scaled_multi_dense_model = MultipleDense()
history = compile_and_fit_with_pinball_loss(scaled_multi_dense_model, conv_window_scaled, 0.1)

valid_performance["Multi step dense"] = scaled_multi_dense_model.evaluate(conv_window_scaled.valid)
performance["Multi step dense"] = scaled_multi_dense_model.evaluate(conv_window_scaled.test, verbose=0)


Epoch 1/1000
1149/1149 [==============================] - 5s 4ms/step - loss: 0.0834 - mean_absolute_error: 0.5255 - mean_squared_error: 0.6060 - _pinball_loss: 0.0834 - val_loss: 0.0524 - val_mean_absolute_error: 0.4219 - val_mean_squared_error: 0.4010 - val__pinball_loss: 0.0524

In [ ]:
scaled_pred_y = scaled_multi_dense_model.predict(conv_window_scaled.test_for_predict)

In [ ]:
_mean = train_df["TARGET"].mean()
_std = train_df["TARGET"].std()
pred_y = (scaled_pred_y * _std + _mean).reshape(-1)

In [ ]:
import numpy as np
true_y = test_df[48:]["TARGET"].values

np.mean(np.maximum(0.1 * (true_y - pred_y), (0.1 - 1) * (true_y - pred_y)))

In [ ]:
from src.preprocessors.preprocessors import inverse_standard_scale

scaled_pred_y = scaled_multi_dense_model.predict(conv_window_scaled.test_for_predict)
pred_y = inverse_standard_scale(train_df["TARGET"], scaled_pred_y)
np.mean(np.maximum(0.1 * (true_y - pred_y), (0.1 - 1) * (true_y - pred_y)))

In [ ]:
from src.model.multiple.convolution import SimpleConv

simple_convolution = SimpleConv()
history = compile_and_fit_with_pinball_loss(simple_convolution, conv_window_scaled, 0.1)

valid_performance["Conv"] = simple_convolution.evaluate(conv_window_scaled.valid)
performance["Conv"] = simple_convolution.evaluate(conv_window_scaled.test, verbose=0)

In [ ]:
import src.model
import importlib
from src.model.loss import pinball_loss_numpy
importlib.reload(src.model)
scaled_pred_y = simple_convolution.predict(conv_window_scaled.test_for_predict)
pred_y = inverse_standard_scale(train_df["TARGET"], scaled_pred_y)
pinball_loss_numpy(true_y, pred_y, 0.1)


In [ ]:
import src.loaders
importlib.reload(src.loaders)

conv_window_scaled.plot()
conv_window_scaled.plot(simple_convolution)